<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>




# 6. 自然語言處理(Natural Language Processing)

在本教程中，我們將從各自獨立的數據片段（如靜態圖像）轉向依賴於序列中其他數據項的資料。在我們的例子中，我們將使用文字語句(text sentences)。語言自然由序列數據組成，以單詞中的字元(characters)和句子中的單詞(word)的形式呈現。序列數據的其他例子包括隨時間變化的股票價格和天氣數據。影片雖然包含靜態圖像，但也是序列。數據中的元素與前後的內容有關聯，這種狀況需要採取不同的方法。

## 6.1 目標(Objectives)

-   使用符記器(tokenizer)準備文本(text)以供神經網路使用
-   了解如何使用內嵌(Embedding)來識別文本數據的數值特徵

## 6.2 BERT



BERT，全名為雙向編碼器表示法轉換器(Bidirectional Encoder Representations from Transformers)，是2018年由[Google](https://www.google.com/)推出的一個突破性模型。

BERT同時針對兩個目標進行訓練：

-   從一系列單詞(sequence of words)中預測缺失的單詞
-   在一系列句子(sequence of sentences)之後預測新句子

讓我們看看BERT如何應對這兩種類型的挑戰。

## 6.3 符記化(Tokenization)

由於神經網路是數字運算機器，讓我們將文本(text)轉換為數值符記(numerical tokens)。讓我們載入BERT的[符記器(tokenizer)](https://huggingface.co/docs/transformers/main_classes/tokenizer#tokenizer)：

### ***bert-base-cased***
一個純英文版本的 BERT詞彙表，而且區分大小寫

In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, BertForQuestionAnswering
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


BERT的`tokenizer`可以一次[編碼(encode)](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.encode)多個文本。我們稍後將測試BERT的記憶力，所以讓我們給它一些資訊和一個關於該資訊的問題。歡迎稍後回到這裡嘗試不同的句子組合。

### ***tokenizer.encode 說明***
會先把文字 切成「符記」(token)，也就是英文的單字、詞根或片段（有些會像是 "quadra", "##tical"）

然後 轉成一串對應的編號（數字）

並且加上 BERT 所需要的「特殊符號」：

    [CLS]：句子的開頭

    [SEP]：每一句話的結尾


In [ ]:
text_1 = "I understand equations, both the simple and quadratical."
text_2 = "What kind of equations do I understand?"

# Tokenized input with special tokens around it (for BERT: [CLS] at the beginning and [SEP] at the end)
#把 text_1 和 text_2 這兩句英文，經過 tokenizer 處理，變成一串數字（token IDs），並自動加上 BERT 需要的特殊符號。
indexed_tokens = tokenizer.encode(text_1, text_2, add_special_tokens=True)
indexed_tokens

[101,
 146,
 2437,
 11838,
 117,
 1241,
 1103,
 3014,
 1105,
 186,
 18413,
 21961,
 1348,
 119,
 102,
 1327,
 1912,
 1104,
 11838,
 1202,
 146,
 2437,
 136,
 102]

上組數字的意思是：

    -101 是 [CLS]
    -後面是第一句 text_1 的編碼
    -102 是 [SEP]
    -接著是第二句 text_2 的編碼
    -最後再一個 102 結尾（結束整個輸入）

如果我們計算符記(token)的數量，會發現符記(token)比我們句子中的單詞更多。讓我們看看為什麼會這樣。我們可以使用[convert_ids_to_tokens](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.convert_ids_to_tokens)來查看使用了哪些符記(token)。

In [ ]:
tokenizer.convert_ids_to_tokens([str(token) for token in indexed_tokens])

['[CLS]',
 'I',
 'understand',
 'equations',
 ',',
 'both',
 'the',
 'simple',
 'and',
 'q',
 '##uad',
 '##ratic',
 '##al',
 '.',
 '[SEP]',
 'What',
 'kind',
 'of',
 'equations',
 'do',
 'I',
 'understand',
 '?',
 '[SEP]']


索引列表(indexed list)比我們原始輸入更長的原因有兩個：

1.  `tokenizer`添加了`special_tokens`來表示序列(sequence)的開始(`[CLS]`)和句子之間的分隔(`[SEP]`)。
2.  `tokenizer`可以將一個單詞分解成多個部分。

從語言學(linguistic)的角度來看，第二點很有趣。許多語言都有[詞根(word roots)](https://en.wikipedia.org/wiki/List_of_Greek_and_Latin_roots_in_English)，或構成單詞的組件。例如，"quadratic"這個詞有詞根"quadr"，意思是"4"。BERT不是使用語言定義的詞根，而是使用[WordPiece](https://paperswithcode.com/method/wordpiece)模型來尋找如何分解單詞的模式。我們今天將使用的BERT模型有`28996`個符記(token)詞彙(vocabulary)。


如果我們想直接[解碼(decode)](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)我們編碼的文本，我們可以做到。注意`special_tokens`已被添加進去。

In [ ]:
tokenizer.decode(indexed_tokens)

'[CLS] I understand equations, both the simple and quadratical. [SEP] What kind of equations do I understand? [SEP]'


## 6.4 文本分段(Segmenting Text)


為了使用BERT模型進行預測，它還需要一個`segment_ids`列表。這是一個與我們的符記(token)長度相同的向量，表示每個segment屬於哪個段落。

由於我們的`tokenizer`添加了一些`special_tokens`，我們可以使用這些特殊符記(token)來找到segment。首先，讓我們定義哪個索引對應哪個特殊符記(token)。

In [ ]:
cls_token = 101
sep_token = 102

接下來，我們可以創建一個`for`迴圈。我們將從`segment_id`設置為`0`開始，每當我們看到[SEP]符記時，我們就增加`segment_id`。為了確保萬無一失，我們將`segment_ids`和`indexd_tokens`都作為張量(tensors)傳回，因為我們稍後會將這些輸入到模型中。

In [ ]:
def get_segment_ids(indexed_tokens):
    segment_ids = []
    segment_id = 0
    for token in indexed_tokens:
        #segment_ids 是我們要建立的「段落 ID 列表」
        #segment_id 是一個計數器，初始值為 0，表示「目前是在第幾句話」
        if token == sep_token:
            segment_id += 1
        segment_ids.append(segment_id)
    segment_ids[-1] -= 1  # Last [SEP] is ignored(忽略最後一個[SEP])
    return torch.tensor([segment_ids]), torch.tensor([indexed_tokens])


讓我們測試一下。每個數字是否正確對應第一句和第二句？

segment_id = 0 表示 第一句（text_1）

segment_id = 1 表示 第二句（text_2）

In [ ]:
segments_tensors, tokens_tensor = get_segment_ids(indexed_tokens)
segments_tensors

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

## 6.4 文本遮罩(Text Masking)


讓我們從BERT對單詞的關注開始。為了訓練詞內嵌(word embeddings)，BERT在一系列單詞中遮蔽(mask out)一個單詞。遮罩(mask)有它自己的特殊符記(token)：

In [ ]:
tokenizer.mask_token

'[MASK]'

In [ ]:
tokenizer.mask_token_id

103


讓我們取之前的兩個句子，遮蔽索引`5`的位置。歡迎回到這裡更改索引以查看結果如何變化！

In [ ]:
masked_index = 5


接下來，我們將應用遮罩並驗證它是否出現在我們的句子序列中。

In [ ]:
indexed_tokens[masked_index] = tokenizer.mask_token_id
tokens_tensor = torch.tensor([indexed_tokens])
tokenizer.decode(indexed_tokens)

'[CLS] I understand equations, [MASK] the simple and quadratical. [SEP] What kind of equations do I understand? [SEP]'


然後，我們將載入用於預測被遮蔽單詞的模型：`modelForMaskedLM`。

從 Hugging Face 載入一個已經訓練好的 BERT 模型，用來做「填空」任務（Masked Language Modeling）

In [ ]:
masked_lm_model = BertForMaskedLM.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).




就像其他PyTorch模組一樣，我們可以檢查架構。

In [ ]:
masked_lm_model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a


你能發現標記為`word_embeddings`的部分嗎？這些是BERT從一個個符記(token)學習到的內嵌(Embedding)。

取得 BERT 模型中「詞嵌入層（word embedding layer）」的參數表，也就是把每個詞轉成向量的「對照表」。

In [ ]:
embedding_table = next(masked_lm_model.bert.embeddings.word_embeddings.parameters())
embedding_table

Parameter containing:
tensor([[-0.0005, -0.0416,  0.0131,  ..., -0.0039, -0.0335,  0.0150],
        [ 0.0169, -0.0311,  0.0042,  ..., -0.0147, -0.0356, -0.0036],
        [-0.0006, -0.0267,  0.0080,  ..., -0.0100, -0.0331, -0.0165],
        ...,
        [-0.0064,  0.0166, -0.0204,  ..., -0.0418, -0.0492,  0.0042],
        [-0.0048, -0.0027, -0.0290,  ..., -0.0512,  0.0045, -0.0118],
        [ 0.0313, -0.0297, -0.0230,  ..., -0.0145, -0.0525,  0.0284]],
       requires_grad=True)


我們可以驗證BERT詞彙表中的`28996`個符記(token)，每一個都有大小為`768`的內嵌(Embedding)或 768 維的向量。

In [ ]:
embedding_table.shape

torch.Size([28996, 768])


讓我們測試模型！它能正確預測我們提供的句子中缺失的單詞(word)嗎？我們將使用[torch.no_grad](https://pytorch.org/docs/stable/generated/torch.no_grad.html)來告知PyTorch不要計算梯度。

In [ ]:
with torch.no_grad():
    predictions = masked_lm_model(tokens_tensor, token_type_ids=segments_tensors)
predictions

MaskedLMOutput(loss=None, logits=tensor([[[ -7.3832,  -7.2504,  -7.4539,  ...,  -6.0597,  -5.7928,  -6.2133],
         [ -6.7681,  -6.7896,  -6.8317,  ...,  -5.4655,  -5.4048,  -6.0683],
         [ -7.7323,  -7.9597,  -7.7348,  ...,  -5.7611,  -5.3566,  -4.3361],
         ...,
         [ -6.1213,  -6.3311,  -6.4144,  ...,  -5.8884,  -4.1157,  -3.1189],
         [-12.3216, -12.4479, -11.9787,  ..., -10.6539,  -8.7396, -11.0487],
         [-13.4115, -13.7876, -13.5183,  ..., -10.6359, -11.6582, -10.9009]]]), hidden_states=None, attentions=None)

這有點難以閱讀，讓我們看看`shape`以更好地了解發生了什麼。

In [ ]:
predictions[0].shape

torch.Size([1, 24, 28996])

`24`是我們的符記(token)數量，`28996`是BERT詞彙表中每個符記(token)的預測。我們想在所有詞彙表符記(token)中找到最高值，所以我們可以使用[torch.argmax](https://pytorch.org/docs/stable/generated/torch.argmax.html)來找到它。

In [ ]:
# Get the predicted token
predicted_index = torch.argmax(predictions[0][0], dim=1)[masked_index].item()
predicted_index

1241


讓我們看看符記(token)`1241`對應的是什麼：

In [ ]:
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
predicted_token

'both'


你覺得呢？是正確的嗎？

In [ ]:
tokenizer.decode(indexed_tokens)

'[CLS] I understand equations, [MASK] the simple and quadratical. [SEP] What kind of equations do I understand? [SEP]'

## 6.5 問與答(Question and Answering)執行閱讀任務


雖然單詞遮罩很有趣，但BERT設計用於更複雜的問題，如句子預測(sentence prediction)。它能夠通過建立在[注意力轉換器(Attention Transformer)](https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)架構上來實現這一點。

我們將在本節中使用不同版本的BERT，它有自己的分詞器(tokenizer)。讓我們為我們的範例句子找到一組新的符記(token)。

In [ ]:
text_1 = "I understand equations, both the simple and quadratical."
text_2 = "What kind of equations do I understand?"

#使用問答專用的 tokenizer，載入一個為問答任務（SQuAD 資料集）特別訓練過的 BERT tokenizer
question_answering_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#編碼兩段文字為 token ID
indexed_tokens = question_answering_tokenizer.encode(text_1, text_2, add_special_tokens=True)

#定義 segment IDs，區分文章(text_1)與問題(text_2)
segments_tensors, tokens_tensor = get_segment_ids(indexed_tokens)


接下來，讓我們載入`question_answering_model`。

In [ ]:
#載入一個已經訓練好、專門用來做問答（Question Answering）任務的 BERT 模型
question_answering_model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



我們可以輸入我們的符記(token)和段落(segments)，就像我們遮蔽一個單詞時一樣。

In [ ]:
# Predict the start and end positions logits
#不需要計算梯度（不用訓練），只是做「推論」
with torch.no_grad():
    out = question_answering_model(tokens_tensor, token_type_ids=segments_tensors)
out

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-5.5943, -4.2960, -5.2682, -1.2511, -6.8350, -0.3992,  2.2274,  2.4654,
         -6.6066,  2.5014, -4.4613, -4.8040, -7.8383, -5.5944, -4.7833, -6.9730,
         -7.1477, -5.2967, -7.4825, -6.7737, -6.8806, -8.6612, -5.5944]]), end_logits=tensor([[-0.7409, -5.3478, -4.2317, -0.0275, -2.6293, -5.9589, -2.8828,  2.7770,
         -4.8512, -2.2092, -2.2413,  4.4412, -0.7181, -0.7411, -3.8988, -5.3865,
         -5.0452, -4.4974, -6.3098, -5.5938, -5.5562, -5.3034, -0.7412]]), hidden_states=None, attentions=None)


`question_answering_model`和回答(answering)模型正在掃描我們輸入的序列資料，以找到最能回答問題的子序列。`start_logits`算出的值越高，答案的段落由此開始的可能性就越大。

In [ ]:
out.start_logits

tensor([[-5.5943, -4.2960, -5.2682, -1.2511, -6.8350, -0.3992,  2.2274,  2.4654,
         -6.6066,  2.5014, -4.4613, -4.8040, -7.8383, -5.5944, -4.7833, -6.9730,
         -7.1477, -5.2967, -7.4825, -6.7737, -6.8806, -8.6612, -5.5944]])


同樣，`end_logits`中的值越高，答案在該符記(token)位置結束的可能性就越大。

In [ ]:
out.end_logits

tensor([[-0.7409, -5.3478, -4.2317, -0.0275, -2.6293, -5.9589, -2.8828,  2.7770,
         -4.8512, -2.2092, -2.2413,  4.4412, -0.7181, -0.7411, -3.8988, -5.3865,
         -5.0452, -4.4974, -6.3098, -5.5938, -5.5562, -5.3034, -0.7412]])


然後我們可以使用[torch.argmax](https://pytorch.org/docs/stable/generated/torch.argmax.html)來透過起始與結束位置得到`answer_sequence`：

In [ ]:
answer_sequence = indexed_tokens[torch.argmax(out.start_logits):torch.argmax(out.end_logits)+1]
answer_sequence

[17718, 23671, 2389]


最後，讓我們[解碼(decode)](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)這些符記(token)，看看答案是否正確！

In [ ]:
#把 token IDs 轉成 token
question_answering_tokenizer.convert_ids_to_tokens(answer_sequence)

['quad', '##ratic', '##al']

In [ ]:
#把 token 組合成一句話
question_answering_tokenizer.decode(answer_sequence)

'quadratical'

#### **** BERT 模型選出 "quadratical"（或包含它的那一段）***

    它在原文中是對「equations」進行分類描述的關鍵詞

    模型認為這段最有可能是對問題「What kind of equations...」的直接回答

    預測方式是根據語意上下文和訓練資料中的問答模式做出的機率推論    

## 6.7 總結(Summary)


做得好！你成功地使用了大型語言模型(Large Language Model, LLM)從一系列句子中提取答案。儘管BERT在首次發布時是最先進的，但許多其他LLM自那以來已經取得了突破。[build.nvidia.com](https://build.nvidia.com/explore/discover)託管了許多這些模型，可以在瀏覽器中進行操作。去看看，看看今天的最先進技術在哪裡！


### 6.7.1 清空記憶體(Clear the Memory)

在繼續之前，請執行以下程式碼區塊(Cell)以清空GPU記憶體。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 6.7.2 下一步(Next)


恭喜，你已完成課程的所有學習目標！

作為最後的練習，並為了獲得課程認證，請在評量(Assessment)中從頭到尾的成功完成圖像分類問題。

<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>


